<a href="https://colab.research.google.com/github/kbw612/Fantasy/blob/main/colab/WeeklyAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

incorrect_player_name_to_correct_player_name_map = {
  "A.J. Dillon": "AJ Dillon",
  "Nathaniel Dell": "Tank Dell",
  "Josh Palmer": "Joshua Palmer",
  "Devon Achane": "De'Von Achane",
  "Gabriel Davis": "Gabe Davis",
  "Devonta Smith": "DeVonta Smith",
  "D.J. Moore": "DJ Moore",
  "Odell Beckham": "Odell Beckham Jr.",
  "Richie James": "Richie James Jr.",
  "Jeff Wilson": "Jeff Wilson Jr.",
  "Jeff Wilson Jr": "Jeff Wilson Jr.",
  "Irv Smith": "Irv Smith Jr.",
  "Irv Smith Jr": "Irv Smith Jr.",
  "Michael Pittman": "Michael Pittman Jr.",
  "Michael Pittman Jr": "Michael Pittman Jr.",
  "Travis Etienne": "Travis Etienne Jr.",
  "Travis Etienne Jr": "Travis Etienne Jr.",
  "Brian Robinson": "Brian Robinson Jr.",
  "Brian Robinson Jr": "Brian Robinson Jr.",
  "Chig Okonkwo": "Chigoziem Okonkwo",
  "DJ Chark Jr": "DJ Chark",
  "DJ Chark Jr.": "DJ Chark",
  "D.J. Chark Jr.": "DJ Chark",
  "Terrace Marshall": "Terrace Marshall Jr.",
  "Terrace Marshall Jr": "Terrace Marshall Jr.",
  "Allen Robinson II": "Allen Robinson",
  "Calvin Austin III": "Calvin Austin",
  "Melvin Gordon III": "Melvin Gordon",
  "Marvin Jones": "Marvin Jones Jr.",
  "Marvin Jones Jr": "Marvin Jones Jr.",
  "N'Keal Harry": "N'Keal Harry",
  "Laviska Shenault": "Laviska Shenault Jr.",
  "Laviska Shenault Jr": "Laviska Shenault Jr.",
  "Kenneth Walker": "Kenneth Walker III",
  "P.J. Walker": "PJ Walker"
}

def clean_player_name(player_name):
  correct_player_name = incorrect_player_name_to_correct_player_name_map.get(player_name, "")

  if correct_player_name == "":
        return player_name

  return correct_player_name

full_team_name_to_team_nickname_map = {
  "Arizona Cardinals": "Cardinals",
  "Atlanta Falcons": "Falcons",
  "Baltimore Ravens": "Ravens",
  "Buffalo Bills": "Bills",
  "Carolina Panthers": "Panthers",
  "Chicago Bears": "Bears",
  "Cincinnati Bengals": "Bengals",
  "Cleveland Browns": "Browns",
  "Dallas Cowboys": "Cowboys",
  "Denver Broncos": "Broncos",
  "Detroit Lions": "Lions",
  "Green Bay Packers": "Packers",
  "Houston Texans": "Texans",
  "Indianapolis Colts": "Colts",
  "Jacksonville Jaguars": "Jaguars",
  "Kansas City Chiefs": "Chiefs",
  "Los Angeles Chargers": "Chargers",
  "Los Angeles Rams": "Rams",
  "Las Vegas Raiders": "Raiders",
  "Miami Dolphins": "Dolphins",
  "Minnesota Vikings": "Vikings",
  "New England Patriots": "Patriots",
  "New Orleans Saints": "Saints",
  "New York Giants": "Giants",
  "New York Jets": "Jets",
  "Philadelphia Eagles": "Eagles",
  "Pittsburgh Steelers": "Steelers",
  "San Francisco 49ers": "49ers",
  "Seattle Seahawks": "Seahawks",
  "Tampa Bay Buccaneers": "Buccaneers",
  "Tennessee Titans": "Titans",
  "Washington Commanders": "Commanders"
}

def get_team_nickname(name):
  team_nickname = full_team_name_to_team_nickname_map.get(name, "")

  if team_nickname == "":
        return name

  return team_nickname

def get_contest_teams(week, start_date, end_date):
  games_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DKSalaries-week{week}.csv")

  games_df = games_df.drop(columns=['Position', 'Name + ID', 'Name', 'ID', 'Roster Position', 'Salary', 'TeamAbbrev', 'AvgPointsPerGame'])

  split_columns = ['Game', 'Date', 'Time', 'TimeZone']
  games_df[split_columns] = games_df['Game Info'].str.split(" ", expand=True)
  games_df = games_df.drop_duplicates(subset=['Game Info', 'Game', 'Date', 'Time', 'TimeZone'])
  games_df = games_df.sort_values(by=['Date', 'Time', 'Game'], ascending=[True, True, True])
  games_df[['Away Team', 'Home Team']] = games_df['Game'].str.split("@", expand=True)

  games_df['Datetime'] = pd.to_datetime(games_df['Date'] + ' ' + games_df['Time'])

  games_df = games_df[(games_df['Datetime'] >= start_date) & (games_df['Datetime'] <= end_date)]
  games_df = games_df.sort_values(by=['Date', 'Time', 'Game'], ascending=[True, True, True])

  split_columns.append('Datetime')
  games_df = games_df.drop(columns=split_columns)

  contest_teams = []
  for index, game_row in games_df.iterrows():
    contest_teams.append(game_row['Away Team'])
    contest_teams.append(game_row['Home Team'])

  return contest_teams

def get_ownership(week):
  # which ownership has all players to be primary df?
  # do anything if cannot find players

  pff_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/ownership-pff-week{week}.csv?1=2")

  for row in pff_df.index:
    if pff_df.at[row, "Pos"] == "D":
      pff_df.at[row, "Player"] = get_team_nickname(pff_df.at[row, "Player"])
    else:
      pff_df.at[row, "Player"] = clean_player_name(pff_df.at[row, "Player"])

    pff_df.at[row, "Own %"] = pd.to_numeric(pff_df.at[row, "Own %"].replace("%", ""), errors='coerce')

  pff_df = pff_df.drop(columns=["ID", "Team", "Opp", "Pos", "Salary"], axis=1).rename(columns={"Player": "PlayerPff", "Own %": "%OwnPff"})
  #,Player,Team,Opp,Pos,Salary,Own %

  ows_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/ownership-ows-week{week}.csv")

  for row in ows_df.index:
    ows_df.at[row, "Player"] = clean_player_name(ows_df.at[row, "Player"])
    ows_df.at[row, "% ownership"] = pd.to_numeric(ows_df.at[row, "% ownership"].replace("%", ""), errors='coerce')

  ows_df = ows_df.drop(columns=["Position", "Team", "Opponent", "Salary", "Notes"], axis=1).rename(columns={"Player": "PlayerOws", "% ownership": "%OwnOws"})
  #Player,Position,Team,Opponent,Salary,% ownership,Notes

  rotowire_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/ownership-rotowire-week{week}.csv?1=2")

  for row in rotowire_df.index:
    if rotowire_df.at[row, "POS"] in["D (DST)", "D"]:
      rotowire_df.at[row, "PLAYER"] = get_team_nickname(rotowire_df.at[row, "PLAYER"])
    else:
      rotowire_df.at[row, "PLAYER"] = clean_player_name(rotowire_df.at[row, "PLAYER"])

  rotowire_df = rotowire_df.drop(columns=["POS", "TEAM", "OPP", "ML", "O/U", "SPRD", "TM/P", "SAL", "FPTS", "VAL"], axis=1).rename(columns={"PLAYER": "PlayerRotowire", "RST%": "%OwnRotowire"})
  #PLAYER,POS,TEAM,OPP,ML,O/U,SPRD,TM/P,SAL,FPTS,VAL,RST%

  ownership_df = pd.merge(pff_df, rotowire_df, left_on="PlayerPff", right_on="PlayerRotowire", how="left")
  ownership_df = ownership_df.drop(columns=["PlayerRotowire"], axis=1)
  ownership_df = pd.merge(ownership_df, ows_df, left_on="PlayerPff", right_on="PlayerOws", how="left")
  ownership_df = ownership_df.drop(columns=["PlayerOws"], axis=1)
  ownership_df = ownership_df.rename(columns={"PlayerPff": "Player"})
  average_ownership = ownership_df[["%OwnOws", "%OwnPff", "%OwnRotowire"]].mean(axis=1)

  #ownership_df = pd.merge(ows_df, rotowire_df, left_on="PlayerOws", right_on="PlayerRotowire", how="left")
  #ownership_df = ownership_df.drop(columns=["PlayerRotowire"], axis=1)
  #ownership_df = ownership_df.rename(columns={"PlayerOws": "Player"})
  #ownership_df["%OwnPff"] = 0

  average_ownership = ownership_df[["%OwnOws", "%OwnRotowire"]].mean(axis=1)
  ownership_df["Avg%Own"] = round(average_ownership, 2)

  #print(ownership_df)

  return ownership_df

def get_injury_report(week):
  injury_report_df = pd.read_csv(f"https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/nfl-injury-report-week{week}.csv?1=2")

  for row in injury_report_df.index:
    injury_report_df.at[row, 'Player'] = clean_player_name(injury_report_df.at[row, 'Player'])

  injury_report_df = injury_report_df.drop(columns=["Team", "Pos", "Injury", "Est. Return"], axis=1)

  return injury_report_df

current_week = 18
nbr_to_print = 1000
contest_start_date = '01/07/2024 01:00PM'
contest_end_date = '01/07/2024 04:30PM'

ownership_df = get_ownership(current_week)
injury_report_df = get_injury_report(current_week)
dk_players_df = pd.read_csv("https://raw.githubusercontent.com/kbw612/Fantasy/main/NFL/2023/DK-Players.csv?1=5")
dk_players_df = pd.merge(dk_players_df, ownership_df, left_on="Name", right_on="Player", how="left")
dk_players_df = pd.merge(dk_players_df, injury_report_df, left_on="Name", right_on="Player", how="left")

contest_teams = get_contest_teams(week=current_week, start_date=contest_start_date, end_date=contest_end_date)

name_column_name = "Name"
position_column_name = 'Position'
current_week_salary_column_name = f"Week{current_week}_Salary"
current_week_expected_fantasy_points_column_name = f"Week{current_week}_Expected"
average_ownership_column_name = "Avg%Own"
ows_ownership_column_name = "%OwnOws"
pff_ownership_column_name = "%OwnPff"
rotowire_ownership_column_name = "%OwnRotowire"
median_fantasy_points_column_name = "Median_FPTS"
average_fantasy_points_column_name = "Average_FPTS"
status_column_name = "Status"

#positions = ["QB", "RB", "WR", "TE"]
contest_teams_condition = dk_players_df["TeamAbbrev"].isin(contest_teams)

filtered_df = dk_players_df[contest_teams_condition]

nbr_of_weeks_to_return = 5
start_week = 1

end_week = 1
if current_week > 1:
  end_week = current_week - 1

if current_week > nbr_of_weeks_to_return:
  start_week = current_week - nbr_of_weeks_to_return

last_weeks = []
for week in range(start_week, end_week):
  last_weeks.append(f"Week{week}_FPTS")

median_FPTS = filtered_df[last_weeks].median(axis=1)
filtered_df[median_fantasy_points_column_name] = round(median_FPTS, 2)

average_FPTS = filtered_df[last_weeks].mean(axis=1)
filtered_df[average_fantasy_points_column_name] = round(average_FPTS, 2)

week_fantasy_points_columns = []
for week in range(end_week, start_week-1, -1):
  #actual_expected_diff_column_name = f"Week{week}_Diff"
  multiplier_column_name = f"Week{week}_Multiplier"
  fantasy_points_column_name = f"Week{week}_FPTS"
  non_td_fantasy_points_column_name = f"Week{week}_Non_TD_FPTS"
  salary_column_name = f"Week{week}_Salary"

  #expected_points = (filtered_df[salary_column_name] * 4) / int(1000)
  filtered_df[multiplier_column_name] = round((filtered_df[fantasy_points_column_name] * 1000) / filtered_df[salary_column_name], 1)
  #filtered_df[actual_expected_diff_column_name] = round(filtered_df[fantasy_points_column_name] - expected_points, 2)
  #week_fantasy_points_columns.append(actual_expected_diff_column_name)
  week_fantasy_points_columns.append(multiplier_column_name)
  week_fantasy_points_columns.append(fantasy_points_column_name)
  if (week >= 12):
    week_fantasy_points_columns.append(non_td_fantasy_points_column_name)

filtered_df[current_week_expected_fantasy_points_column_name] = round((filtered_df[current_week_salary_column_name] * 4) / int(1000), 2)

sorted_df = filtered_df.sort_values(by=[current_week_salary_column_name], ascending=[False])
sorted_columns = [name_column_name, status_column_name, position_column_name, current_week_salary_column_name, current_week_expected_fantasy_points_column_name, average_ownership_column_name, ows_ownership_column_name, pff_ownership_column_name, rotowire_ownership_column_name, median_fantasy_points_column_name, average_fantasy_points_column_name]
sorted_columns.extend(week_fantasy_points_columns)
sorted_df = sorted_df[sorted_columns].head(nbr_to_print)

rename_column_names = {median_fantasy_points_column_name: "Median",
                       average_fantasy_points_column_name: "Avg",
                       average_ownership_column_name: "Avg %",
                       ows_ownership_column_name: "Ows %",
                       pff_ownership_column_name: "PFF %",
                       rotowire_ownership_column_name: "RW %"}
rename_column_names[current_week_salary_column_name] = f"W{current_week}"
rename_column_names[current_week_expected_fantasy_points_column_name] = f"E{current_week}"
for week in range(end_week, start_week-1, -1):
  #rename_column_names[f"Week{week}_Diff"] = f"D{week}"
  rename_column_names[f"Week{week}_Multiplier"] = f"M{week}"
  rename_column_names[f"Week{week}_FPTS"] = f"P{week}"
  if (week >= 12):
    rename_column_names[f"Week{week}_Non_TD_FPTS"] = f"NTDP{week}"

sorted_df.rename(columns=rename_column_names, inplace=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#lineup_df.to_string(justify='left', index=False)
sorted_df[name_column_name] = sorted_df[name_column_name].apply(lambda x: x.ljust(2))
pd.set_option('display.max_colwidth', 20)

sorted_df.to_csv(f"DK-Main-Slate-Content-week-{current_week}.csv", index=False)

with pd.option_context("display.max_rows", nbr_to_print):
  print(ownership_df.to_string(index=False))
  print("")



                  Player %OwnPff  %OwnRotowire %OwnOws  Avg%Own
             CeeDee Lamb    35.7         14.16    32.7    23.43
          Christian Kirk    28.2          6.61     NaN     6.61
            Jordan Mason    27.7          4.72    33.2    18.96
           Ronnie Rivers    26.3          2.54     5.8     4.17
        Justin Jefferson    19.3         13.06    32.5    22.78
            Dak Prescott    18.1          7.52    13.2    10.36
            Trey McBride    17.5          6.75    16.8    11.78
              DK Metcalf    15.8          6.41     9.4     7.90
                DJ Moore    15.4          7.47     3.6     5.54
              A.J. Brown    15.1          8.08     4.6     6.34
               Noah Gray    14.6          2.73    11.3     7.02
             Zamir White    13.9         14.19    22.4    18.29
       Amon-Ra St. Brown    12.8          5.91    14.9    10.40
           Davante Adams    12.6          9.23    13.3    11.26
              Geno Smith    11.1        

<ipython-input-1-2ebfb444c64e>:229: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[median_fantasy_points_column_name] = round(median_FPTS, 2)
<ipython-input-1-2ebfb444c64e>:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[average_fantasy_points_column_name] = round(average_FPTS, 2)
<ipython-input-1-2ebfb444c64e>:243: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 